In [2]:
import lsst.meas.astrom as measAstrom
import lsst.daf.base as dafBase
import lsst.pipe.base as pipeBase
import lsst.daf.persistence as dafPersist

import random
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.display.ds9 as ds9
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom


import DiaSourceTools as DSTools
import lsst.afw.display.utils as displayUtils
import lsst.meas.algorithms.detection as sDet

%matplotlib inline

/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")

In [4]:
#visitref = 800719
#visitlist = [800721]
visitref = (845345, 4)
visitlist = [(849373,20), (849686,24), (850180,28), (850586,30+2), (851060,30+6)]


dataId = {'visit': visitref[0], 'filter':'r' , 'ccd':14}
diaSrcref = butler.get("deepDiff_diaSrc", dataId)
Srcref = butler.get("deepDiff_diaSrc", dataId)
schema = diaSrcref.schema
newsrclist=[(diaSrcref,visitref)]
objlist = []

for visit in visitlist:    
    dataId = {'visit': visit[0], 'filter':'r' , 'ccd':14}
    diaSrc= butler.get("deepDiff_diaSrc", dataId)
    match = afwTable.matchRaDec(diaSrcref, diaSrc, afwGeom.Angle(1./3600., afwGeom.degrees))        
    newSrc = afwTable.SourceCatalog(schema)
    print len(match)
    for m in match:
        m.second.setId(m.first.getId())
        newSrc.append(m.second)
        if not(m.first.getId() in objlist):
            objlist.append(m.first.getId())
    newsrclist.append((newSrc,visit))
    print "-----",len(objlist)


58
----- 58
110
----- 151
112
----- 234
119
----- 302
167
----- 415


In [5]:
len(diaSrcref['id'])

1338

In [6]:
newSrc = newSrc.copy(deep=True)

In [7]:
#print newSrc['id']


In [8]:
#newSrc.writeFits("hop.fits")

In [9]:
import sncosmo

In [10]:
def visualize_lightcurve(lightcurve, color='black'):
    """
    Make a simple light curve plot.
    """
    if lightcurve is None:
        raise ValueError('No lightcurve yet. Use build_lightcurve first.')

    fig = sncosmo.plot_lc(lightcurve, color=color)

    return fig


In [11]:
from astropy.table import Table
def build_lightcurve(newSrcList,objid):
    """
    Assemble a light curve data table from available files.
    """
        
    bandpasses = ['r']
    visit_list = ['800719']
        
    lightcurve = {}
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    lightcurve['ra'] = []
    lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []

    for bandpass in bandpasses:
        #for visit in self.visit_list:
        for (newSrc,visit) in newSrcList:    
            #dataId = {'visit': visit, 'filter':bandpass , 'ccd':14}
            #print visit
            for src in newSrc:
                if (src['id'] == objid):
                    #print 'yep',visit
                    lightcurve['bandpass'].append(str('sdss' + bandpass))
                    lightcurve['mjd'].append(visit[1])
                    lightcurve['ra'].append(src['coord_ra'])
                    lightcurve['dec'].append(src['coord_dec'])
                    lightcurve['flux'].append(src['base_PeakLikelihoodFlux_flux'])
                    lightcurve['flux_error'].append(src['base_PeakLikelihoodFlux_fluxSigma'])
                    #lightcurve['flux'].append(src['base_CircularApertureFlux_12_0_flux'])
                    #lightcurve['flux_error'].append(src['base_CircularApertureFlux_12_0_fluxSigma'])
                    lightcurve['zp'].append(25.0)
                    lightcurve['zpsys'].append('ab')
    lightcurve = Table(data=lightcurve)
    return lightcurve    



In [13]:
newsrclist,obj


([(<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7f956e750930> >,
   (845345, 4)),
  (<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7f956e763600> >,
   (849373, 20)),
  (<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7f956e755660> >,
   (849686, 24)),
  (<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7f956e750d50> >,
   (850180, 28)),
  (<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::afw::table::SortedCatalogT< lsst::afw::table::SourceRecord > *' at 0x7f956e7556f0> >,
   (850586, 32)),
  (<lsst.afw.table.tableLib.SourceCatalog; proxy of <Swig Object of type 'lsst::a

In [12]:
i=0
lclist=[]
print len(objlist)
for obj in objlist:
    i = i+1
    print i
    lc = build_lightcurve(newsrclist,obj)
    if len(lc)>1:
        lclist.append(lc)


415
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


RuntimeError: Field 'id' does not have the given type.

In [ ]:
i=0
import math
for lc in lclist:
    #print len(lc)
    if len(lc)>3:
        i=i+1
        dra = float(lc['ra'][0])-3.7472052
        ddec = float(lc['dec'][0])-0.92077529
        print math.sqrt(dra*dra+ddec*ddec)/3.14159*180*3600,float(lc['ra'][0])/3.14159*180,float(lc['dec'][0])/3.14159*180
        visualize_lightcurve(lc)
print i 
lc   

In [ ]:
import lsst.afw.geom as afwGeom
import lsst.afw.display.utils as displayUtils
import matplotlib.pyplot as plt
import numpy as np
import lsst.afw.table as afwTable
from lsst.meas.algorithms.detection import SourceDetectionTask
import lsst.meas.algorithms.detection as sDet

def get_time_mosaic(butler, dataid_list, source, frame=1):
    mosaic = displayUtils.Mosaic(gutter=5, background=3, mode="x")
    
    
    
    for dataid in dataid_list:
        
        mosaic_temp = displayUtils.Mosaic(gutter=0, background=0, mode="y")
        
        diffExp = butler.get("deepDiff_differenceExp", dataid)
        sciExp  = butler.get("calexp", dataid)
        tmpExp = butler.get("deepDiff_warpedExp", dataid)
        bgConf = sDet.BackgroundConfig()
        background,tmpExp = sDet.estimateBackground(tmpExp,bgConf,True)
        
        s1 = get_stamp(source, sciExp)
        s2 = get_stamp(source, tmpExp)
        s3 = get_stamp(source, diffExp)
        
        mosaic_temp.append(s1.getMaskedImage())
        mosaic_temp.append(s2.getMaskedImage())
        mosaic_temp.append(s3.getMaskedImage())
        m = mosaic_temp.makeMosaic(frame=None, display=None).clone()
        mosaic.append(m)
        
    mosaic.makeMosaic(frame=frame, title="time mosaic")

def get_stamp(source, exposure, offset=10):

    
    bbox = source.getFootprint().getBBox()
    mos = displayUtils.Mosaic()
                
    sourceRa = source.getRa()
    sourceDec = source.getDec()

    wcs = exposure.getWcs()
    mos = displayUtils.Mosaic()


    Center = afwGeom.Point2I(wcs.skyToPixel(sourceRa, sourceDec))

    Begin = afwGeom.Point2D(Center.getX() - bbox.getHeight()/2., Center.getY() - bbox.getHeight()/2.)
    Begin = afwGeom.Point2I(Begin)
    End = afwGeom.Point2D(Center.getX() + bbox.getHeight()/2., Center.getY() + bbox.getHeight()/2.)
    End = afwGeom.Point2I(End)



    ExpOrig = afwGeom.Point2I(exposure.getX0()-1, exposure.getY0()-1)



    correctedBegin = Begin- ExpOrig
    correctedEnd = End - ExpOrig

    correctedBegin= afwGeom.Point2I(correctedBegin.getX()-offset,correctedBegin.getY()-offset )
    correctedEnd = afwGeom.Point2I(correctedEnd.getX()+offset,correctedEnd.getY()+offset )
    bboxT = afwGeom.Box2I(correctedBegin,correctedEnd) 

    #print bboxT.toString
    return exposure.Factory(exposure,bboxT, True)


In [ ]:
source = Srcref[140]
dataId = {'visit': visitref[0], 'filter':'r' , 'ccd':14}
dataid_list=[dataId]
for visit in visitlist:
    print visit[0]
    dataId = {'visit': visit[0], 'filter':'r' , 'ccd':14}
    dataid_list.append(dataId)

get_time_mosaic(butler, dataid_list, source, frame=1)

In [ ]:
i=0
import math
for lc in lclist:
    #print len(lc)
    if len(lc)>3:
        i=i+1
        ra = float(lc['ra'][0])
        dec =  float(lc['dec'][0])
        for s in Srcref:
            r=math.sqrt((float(s['coord_ra'])-ra)**2+(float(s['coord_dec'])-dec)**2)/3.14159*180*3600
            if (r<1) : 
                get_time_mosaic(butler, dataid_list, s, frame=i) 
                
print i 
lc   

In [ ]:
print len(Srcref)

In [ ]:
candidates = []
i=0
for lc in lclist:
    #print len(lc)
    if len(lc)>3:
        i=i+1
        ra = float(lc['ra'][0])
        dec =  float(lc['dec'][0])
        for s in Srcref:
            r=math.sqrt((float(s['coord_ra'])-ra)**2+(float(s['coord_dec'])-dec)**2)/3.14159*180*3600
            if (r<1) : 
                candidates.append(s)

In [ ]:
for c in candidates:
    print c.getRa(), c.getDec()

In [ ]:
dataId = dataid_list[4]
final = []
print dataId
diffExp = butler.get("deepDiff_differenceExp", dataId)
i = 0
for c in candidates:
    stamp = DSTools.get_stamp(c, diffExp)
    stamp_clone = stamp.clone()
    
    results = DSTools.detect_diasources(stamp_clone, doSmooth=True)
    negFps = len(results.fpSets.negative.getFootprints())
    posFps = len(results.fpSets.positive.getFootprints())
    pos_flux_at_25 = get_flux_percentage_at(stamp)
    neg_flux_at_25 = get_flux_percentage_at(stamp, plane_mask="DETECTED_NEGATIVE")
  
    if negFps == 0:
        DSTools.get_time_mosaic(butler, dataid_list, c, frame=i)
        final.append(c)
        i+=1

In [ ]:
candidates

In [ ]:
f = final[-1]
f['coord_ra']

In [ ]:
candidates = []
i=0
for lc in lclist:
    #print len(lc)
    if len(lc)>3:
        i=i+1
        ra = float(lc['ra'][0])
        dec =  float(lc['dec'][0])
        for f in final:
            r=math.sqrt((float(f['coord_ra'])-ra)**2+(float(f['coord_dec'])-dec)**2)/3.14159*180*3600
            if (r<1) :
                visualize_lightcurve(lc, color='red')

In [ ]:
sncosmo.plot_lc??

In [ ]:
def get_flux_percentage_at(stamp, percent = 0.25, plane_mask="DETECTED", positive=True):
    
    cumsum = DSTools.get_cumulative_flux(stamp, plane_mask=plane_mask, positive=positive)
    
    if cumsum is None:
        return None
    
    total = float(len(cumsum))
    
    for i, value in enumerate(cumsum):
        #print i, float(i+1)/total, 1-value
        if float(i+1)/total > (1-percent):
            return 1-value